# Image Stack Viewer Workflow

<div style="text-align: center;">
    <img src="./assets/230620_video-viewer.png" alt="video viewer preview" width="450"/>
</div>

## Summary
This workflow is dedicated to visualizing Miniscope calcium imaging data. It's designed for handling large datasets typical in neuroscience, especially single-photon miniscope data, providing an efficient visualization tool with features tailored to the needs of this field.

## Setup and Imports

<div class="admonition alert alert-info">
<p><strong> Dependencies</strong></p>

This workflow notebook requires the packages specified in the [environment file](./environment.yml) in this workflow directory.
</div>


In [1]:
import os
import sys
from functools import partial

import numpy as np
import xarray as xr
import panel as pn
import holoviews as hv

from neurodatagen.ca_imaging import simulate_miniscope_data
from holoviews.streams import Stream
from hvplot import xarray

# Configuration
%load_ext autoreload
%autoreload 2

# Extensions
pn.extension(throttled=False)
hv.extension('bokeh')

## Data Handling Overview

Effective data handling is crucial for neuroscience imaging. This section outlines our approach to managing both synthetic and real data, highlighting the key differences and our strategies for processing them.


### Generating Synthetic Data

Synthetic data generation is an essential step for testing, developing, and demonstrating visualization tools. We use simulated data to mimic the properties of real miniscope data, ensuring our tools are reliable and accurate.

The `simulate_miniscope_data` function used below generates synthetic calcium imaging data by simulating neural activity through spatial footprints and temporal calcium traces, while incorporating motion artifacts and realistic background noise. The function outputs a numpy array of 8-bit unsigned integers, encapsulated within an `xarray.DataArray`. This array represents the generated imaging data with dimensions corresponding to frame height, frame width, and frame number.

In [ ]:
ncell = 50
dims = {'height': 512, 'width': 512, 'frame': 1000}
arr_name='sim_miniscope'
chk_size = 200 # chunk size of frames
dpath = '~/data/miniscope_simulated.zarr' # location to be stored to or loaded from
chunks = {'frame': chk_size} # chunk by set of complete frames

data = simulate_miniscope_data(ncell=ncell, dims=dims, arr_name=arr_name, chk_size=chk_size)
data

# **TODO** set chunks here
# **TODO:** add option to generate data straight to disk

#### Write

In [ ]:
# Save the data to disk to mimic a common workflow

write = True # turn on to save write data
if write:
    data.to_zarr(dpath, mode='w') # parallel storage of the different output chunks

#### Read

In [ ]:
ldataset = xr.open_dataset(dpath, engine='zarr', chunks=chunks) # TODO: why need to specify chunks on read? should use zarr chunking by default
data = ldataset[arr_name]
data

### Visualize synthetic data

#### Version 1: Simple viewer, hvplot, default slider

In [ ]:
data.hvplot.image(groupby="frame", cmap="Viridis", height=400, width=400, colorbar=False)

#### Version 1.1: Simple viewer, hvplot interactive, default slider, no groupby

This version avoids the use of `groupby` on a dask array because this can sometimes be a costly operation, see [this.](https://docs.xarray.dev/en/latest/user-guide/dask.html#dask:~:text=More%20generally%2C%20groupby()%20is%20a%20costly%20operation%20and%20will%20perform%20a%20lot%20better%20if%20the%20flox%20package%20is%20installed.)

In [ ]:
frames = data.coords["frame"].values
slider = pn.widgets.IntSlider(start=int(frames.min()), end=int(frames.max()))
frame = data.interactive().isel(frame=slider)
frame.hvplot.image(cmap="Viridis", height=400, width=400, colorbar=False)

#### Version 2: Intermediate viewer, hvPlot, Player Widget, frame dim views

In [ ]:
fs = 30 # frames per second
frames = data.coords["frame"].values

video_player = pn.widgets.Player(
    length=len(data.coords["frame"]),
    interval=1000//fs, # ms per frame
    value=int(frames.min()),
    max_width=400,
    max_height=90,
    loop_policy="loop",
    sizing_mode="stretch_width",
)

main_h_w = data.hvplot.image(
    groupby="frame",
    cmap="Viridis",
    frame_height=400,
    frame_width=400,
    colorbar=False,
    widgets={"frame": video_player},
)

def update_vline(value):
    return hv.VLine(value)

def update_hline(value):
    return hv.HLine(value)

dmap_vline = hv.DynamicMap(update_vline, streams=[video_player.param.value]).opts(color='red', alpha=.6, line_width=3)
dmap_hline = hv.DynamicMap(update_hline, streams=[video_player.param.value]).opts(color='red', alpha=.6, line_width=3)

height_frame_neighbor = data.mean(['width']).hvplot.image(x='frame',
    cmap="Viridis",
    frame_height=400,
    frame_width=200,
    colorbar=False,
    title='_', # hack to ensure that xlabel is rendered, see https://github.com/bokeh/bokeh/issues/13225#issuecomment-1611172355
) * dmap_vline

# Ideally, we want the frame_width_neighbor to have yaxis inverted so it progresses away from main like height_frame_neighbor
frame_width_neighbor = data.mean(['height']).hvplot.image(y='frame',
    cmap="Viridis",
    frame_height=200,
    frame_width=400,
    colorbar=False,
) * dmap_hline # invert_yaxis=True doesn't work here, see https://github.com/holoviz/holoviews/issues/5801
    # ylim = (data.frame.size-1,0), # this is supposed to be a inver_yaxis workaround but doesn't work here, from https://discourse.holoviz.org/t/hvplot-image-invert-y-axis/5625/2?u=droumis

video_player.margin = (20, 20, 20, 70) # hack to center widget over main
# main_h_w[0] below selects just the plot so we can handle the widget placement explicitly
pn.Column(video_player, pn.Row(main_h_w[0], height_frame_neighbor), frame_width_neighbor)

## Real data pipeline

### Intake data

#### The minian repository includes some demo data. 

**Option 1 (recommended):** Clone from the forked source:

1. `cd <location where minian should be downloaded>`
2. `git clone git@github.com:droumis/minian.git`

Now you have about 722 MB of .avi files in `minian/demo_movies` and another `10 MB` of .nc files in `minian/demo_data`. Note, this is a fork of the real repository with (currently) minor changes to work with latest xarray version.

**Option 2:** follow [conda install instructions](https://minian.readthedocs.io/en/stable/start_guide/install.html#install-using-conda), then run [`minian-install --demo`](https://minian.readthedocs.io/en/stable/start_guide/install.html#getting-notebooks-and-demos). Note, there is no conda installable package for minian on osx_arm64 (M1) macos systems so option 1 is highly recommended in this context. The remaining steps, like setting `sys.path.append` assume **option 1** was followed, otherwise just import mininan.

In [ ]:
dpath = "~/src/minian/demo_movies/"
dpath = os.path.expanduser(dpath)

In [ ]:
minian_path = "~/src/minian/"
minian_path = os.path.expanduser(minian_path)
minian_path

In [ ]:
sys.path.append(minian_path)

In [ ]:
from minian.utilities import (
    TaskAnnotation,
    get_optimal_chk,
    load_videos,
    open_minian,
    save_minian,
)

The load_videos function is loading a series of .avi files as an xarray DataArray backed by Dask arrays. This means that the video data is not immediately loaded into memory; instead, Dask creates a plan (task graph) for how to load the data when necessary. This is beneficial when working with large datasets that don't fit into memory, as it allows you to perform computations in a lazy manner.

In [ ]:
varr = load_videos(dpath, pattern='msCam[0-9]+\\.avi$')

In [ ]:
varr = varr.rename("varr")
varr

In [ ]:
# chk, _ = get_optimal_chk(varr, dtype=float)

# chk["height"] = -1
# chk["width"] = -1

# chk

In [ ]:
chk_size = 100 # chunk size of frames
chunks = {'frame': chk_size, "height":-1, "width":-1} # chunk by set of complete frames

chunks

We then immediately save the array representation to the intermediate folder to avoid repeatedly loading the video in later steps.

In [ ]:
intpath = "~/src/minian/minian_intermediate"
intpath = os.path.expanduser(intpath)
intpath

In [ ]:
%%time
varr = save_minian(varr, intpath, overwrite=True, chunks=chunks)

In [ ]:
varr

In [ ]:
%%capture
varr.hvplot.image(groupby="frame", cmap="Viridis", height=400, width=400, colorbar=False)

In [ ]:
frames = varr.coords["frame"].values
slider = pn.widgets.IntSlider(start=int(frames.min()), end=int(frames.max()))
frame = varr.interactive().isel(frame=slider)
frame.hvplot.image(cmap="Viridis", height=400, width=400, colorbar=False)

In [ ]:
fs = 30 # frames per second
frames = data.coords["frame"].values

video_player = pn.widgets.Player(
    length=len(data.coords["frame"]),
    interval=1000//fs, # ms per frame
    value=int(frames.min()),
    max_width=400,
    max_height=90,
    loop_policy="loop",
    sizing_mode="stretch_width",
)

main_h_w = data.hvplot.image(
    groupby="frame",
    cmap="Viridis",
    frame_height=400,
    frame_width=400,
    colorbar=False,
    widgets={"frame": video_player},
)

def update_vline(value):
    return hv.VLine(value)

def update_hline(value):
    return hv.HLine(value)

dmap_vline = hv.DynamicMap(update_vline, streams=[video_player.param.value]).opts(color='red', alpha=.6, line_width=3)
dmap_hline = hv.DynamicMap(update_hline, streams=[video_player.param.value]).opts(color='red', alpha=.6, line_width=3)

height_frame_neighbor = data.mean(['width']).hvplot.image(x='frame',
    cmap="Viridis",
    frame_height=400,
    frame_width=200,
    colorbar=False,
    title='_', # hack to ensure that xlabel is rendered, see https://github.com/bokeh/bokeh/issues/13225#issuecomment-1611172355
) * dmap_vline

# Ideally, we want the frame_width_neighbor to have yaxis inverted so it progresses away from main like height_frame_neighbor
frame_width_neighbor = data.mean(['height']).hvplot.image(y='frame',
    cmap="Viridis",
    frame_height=200,
    frame_width=400,
    colorbar=False,
    # ylim = (data.frame.size-1,0), # this is supposed to be a inver_yaxis workaround but doesn't work here, from https://discourse.holoviz.org/t/hvplot-image-invert-y-axis/5625/2?u=droumis
).opts(invert_yaxis=False) * dmap_hline # invert_yaxis=True doesn't work here, see https://github.com/holoviz/holoviews/issues/5801

video_player.margin = (20, 20, 20, 70) # hack to center widget over main
# main_h_w[0] below selects just the plot so we can handle the widget placement explicitly
pn.Column(video_player, pn.Row(main_h_w[0], height_frame_neighbor), frame_width_neighbor)

## Other viz implementations to ignore for now

### Version 3: Intermediate viewer: HoloViews, frame-stream, Player

In [ ]:
frames = data.coords["frame"].values
f_min = int(frames.min())
f_max = int(frames.max())
height = data.sizes["height"]
width = data.sizes["width"]
fs = 30 # frames per second

# Generate Image object for a given frame
def generate_image(frame, data):  
    return hv.Image(
        data.sel(frame=frame).compute(), kdims=["width", "height"]
    )

# Setup frame stream
frame_param = param.Integer(default=f_min, bounds=(f_min, f_max))
FrameStream = Stream.define("FrameStream", frame=frame_param)
frame_stream = FrameStream()

# Dynamic map of image via frame stream
image_generator = pn.bind(generate_image, data=data)
image_map = hv.DynamicMap(image_generator, streams=[frame_stream]).opts(
    frame_width=500, aspect=width / height, cmap="Viridis"
)

# Create a video player widget
video_player = pn.widgets.Player(
    length=len(frames), interval=1000//fs, value=f_min, width=600, height=90
)

# update the frame stream when a new event occurs on the widget
def update_frame_stream(event):
    frame_stream.event(frame=int(frames[event.new]))

# Link player widget to the frame stream update function
video_player.param.watch(update_frame_stream, "value")

layout = pn.layout.Column(video_player, image_map)
layout.servable() # serve the app if this notebook is run on the command line with panel serve <notebook>

### Version 4: Advanced Viewer (adapted Minian viewer): Class, clr hist, mask selection, summary stats, input types

In [ ]:
# Separate imports because this cell probably won't stay in this workflow notebook

from typing import Union, List, Optional
import numpy as np
import xarray as xr
import holoviews as hv
import panel as pn
import param
import functools as fct
import itertools as itt
from collections import OrderedDict
from datashader import count_cat
from holoviews.streams import Stream, BoxEdit, RangeXY
from holoviews.operation.datashader import datashade
import holoviews as hv; hv.extension('bokeh')
import panel.widgets as pnwgt
from bokeh.palettes import Category10_10


class VArrayViewer:
    """
    Interactive visualization for movie data arrays.
    
    Parameters
    ----------
    varr : Union[xr.DataArray, List[xr.DataArray], xr.Dataset]
        Input array, list of arrays, or dataset to be visualized. Each array
        should contain dimensions "height", "width" and "frame". If a
        dataset, then the dimensions specified in `meta_dims` will be used
        as metadata dimensions that can uniquely identify each array. If a
        list, then a dimension "data_var" will be constructed and used as
        metadata dimension, and the `.name` attribute of each array will be
        used to identify each array.
    framerate : int, optional
        The framerate of playback when using the toolbar. By default `30`.
    summary : list, optional
        List of summary statistics to plot. The statistics should be one of
        `{"mean", "max", "min", "diff"}`. By default `["mean", "max"]`.
    meta_dims : List[str], optional
        List of dimension names that can uniquely identify each input array
        in `varr`. Only used if `varr` is a `xr.Dataset`. By default `None`.
    datashading : bool, optional
        Whether to use datashading on the summary statistics. By default
        `True`.
    layout : bool, optional
        Whether to visualize all arrays together as layout. If `False` then
        only one array will be visualized and user can switch array using
        drop-down lists below the *Play Toolbar*. By default `False`.

    Raises
    ------
    NotImplementedError
        if `varr` is not a `xr.DataArray`, a `xr.Dataset` or a list of `xr.DataArray`
    
    More:
    ------
    The visualization contains following panels from top to bottom:

    Play Toolbar
        A toolbar that controls playback of the video. Additionally, when the
        button "Update Mask" is clicked, the coordinates of the box drawn in
        *Current Frame* panel will be used to update the `mask` attribute of the
        `VArrayViewer` instance, which can be later used to subset the data. If
        multiple arrays are visualized and `layout` is `False`, then drop-down
        lists corresponding to each metadata dimensions will show up so the user
        can select which array to visualize.
    Current Frame
        Images of the current frame. If multiple movie array are passed in,
        multiple frames will be labeled and shown. To the side of each frame
        there is a histogram of intensity values. The "Box Select" tool can be
        used on the histogram to limit the range of intensity used for
        color-mapping. Additionally, the "Box Edit Tool" is available for use on
        the frame image, where you can hold "Shift" and draw a box, whose
        coordinates can be used to update the `mask` attribute of the
        `VarrayViewer` instance (remember to click "Update Mask" after drawing).
    Summary
        Summary statistics of each frame across time. Only shown if `summary` is
        not empty. The red vertical line indicate current frame.

    Attributes
    ----------
    mask : dict
        Instance attribute that can be retrieved and used to subset data later.
        Keys are `tuple` with values corresponding to each `meta_dims` and
        uniquely identify each input array. If `meta_dims` is empty then keys
        will be empty `tuple` as well. Values are `dict` mapping dimension names
        (of the arrays) to subsetting slices. The slices are in the plotting
        coorandinates and can be directly passed to `xr.DataArray.sel` method to
        subset data.
    """

    def __init__(
        self,
        varr: Union[xr.DataArray, List[xr.DataArray], xr.Dataset],
        framerate=30,
        summary=["mean", "max"],
        meta_dims: List[str] = None,
        datashading=True,
        layout=False,
    ):
        # Handling different types of `varr` input
        if isinstance(varr, list):
            # If `varr` is a list, assign a new coordinate `data_var` using 
            # the name of each array and concatenate them along the `data_var` dimension
            for iv, v in enumerate(varr):
                varr[iv] = v.assign_coords(data_var=v.name)
            self.ds = xr.concat(varr, dim="data_var")
            meta_dims = ["data_var"]
        elif isinstance(varr, xr.DataArray):
            # If `varr` is a DataArray, convert it into a Dataset
            self.ds = varr.to_dataset()
        elif isinstance(varr, xr.Dataset):
            # If `varr` is a Dataset, keep it as it is
            self.ds = varr
        else:
            # If `varr` is not a list, DataArray, or Dataset, raise an error
            raise NotImplementedError(
                "video array of type {} not supported".format(type(varr))
            )

        # Initialize metadata based on the specified metadata dimensions
        try:
            self.meta_dicts = OrderedDict(
                [(d, list(self.ds.coords[d].values)) for d in meta_dims]
            )
            self.cur_metas = OrderedDict(
                [(d, v[0]) for d, v in self.meta_dicts.items()]
            )
        except TypeError:
            self.meta_dicts = dict()
            self.cur_metas = dict()

        # Initialize some attributes
        self._datashade = datashading
        self._layout = layout
        self.framerate = framerate
        self._f = self.ds.coords["frame"].values
        self._h = self.ds.sizes["height"]
        self._w = self.ds.sizes["width"]
        self.mask = dict()

        # Define streams for interaction
        CStream = Stream.define(
            "CStream",
            f=param.Integer(
                default=int(self._f.min()), bounds=(self._f.min(), self._f.max())
            ),
        )
        self.strm_f = CStream()
        self.str_box = BoxEdit()
        self.widgets = self._widgets()

        # Compute and store summary statistics
        if type(summary) is list:
            summ_all = {
                "mean": self.ds.mean(["height", "width"]),
                "max": self.ds.max(["height", "width"]),
                "min": self.ds.min(["height", "width"]),
                "diff": self.ds.diff("frame").mean(["height", "width"]),
            }
            try:
                summ = {k: summ_all[k] for k in summary}
            except KeyError:
                print("{} Not understood for specifying summary".format(summary))
            if summ:
                # print("computing summary")
                sum_list = []
                for k, v in summ.items():
                    sum_list.append(v.compute().assign_coords(sum_var=k))
                summary = xr.concat(sum_list, dim="sum_var")
        self.summary = summary

        # Initialize subsets of data and summary statistics based on layout option
        if layout:
            self.ds_sub = self.ds
            self.sum_sub = self.summary
        else:
            self.ds_sub = self.ds.sel(**self.cur_metas)
            try:
                self.sum_sub = self.summary.sel(**self.cur_metas)
            except AttributeError:
                self.sum_sub = self.summary

        # Generate the panel plot
        self.pnplot = pn.panel(self.get_hvobj())

    def get_hvobj(self):
        """
        Generates a holoviews Layout object of the image stack
        """
        def get_im_ovly(meta):  # Function to generate overlay of image and box
            def img(f, ds):  # Function to generate HoloViews image object for a given frame
                return hv.Image(ds.sel(frame=f).compute(), kdims=["width", "height"])

            # Select a sub-dataset based on metadata; if not possible, use the original dataset
            try:
                curds = self.ds_sub.sel(**meta).rename("_".join(meta.values()))
            except ValueError:
                curds = self.ds_sub

            fim = fct.partial(img, ds=curds)  # Partial function for image generation with current dataset

            # Create a dynamic map for images with the given partial function and frame stream
            im = hv.DynamicMap(fim, streams=[self.strm_f]).opts(
                frame_width=500, aspect=self._w / self._h, cmap="Viridis"
            )

            # Define a range of x and y coordinates for the image
            self.xyrange = RangeXY(source=im).rename(x_range="w", y_range="h")

            # Create a box if layout is not yet defined
            if not self._layout:
                hv_box = hv.Polygons([]).opts(
                    fill_alpha= 0.3, line_color= "white"
                )
                self.str_box = BoxEdit(source=hv_box)
                im_ovly = im * hv_box  # Create an overlay of the image and the box
            else:
                im_ovly = im  # If layout already defined, use the image as is

            def hist(f, w, h, ds):  # Function to generate histogram for given frame, width, height and dataset
                if w and h:
                    cur_im = hv.Image(
                        ds.sel(frame=f).compute(), kdims=["width", "height"]
                    ).select(height=h, width=w)
                else:
                    cur_im = hv.Image(
                        ds.sel(frame=f).compute(), kdims=["width", "height"]
                    )
                return hv.operation.histogram(cur_im, num_bins=50).opts(
                    xlabel="fluorescence", ylabel="freq"
                )

            fhist = fct.partial(hist, ds=curds)  # Partial function for histogram generation with current dataset

            # Create a dynamic map for histograms with the given partial function and frame & xy range streams
            his = hv.DynamicMap(fhist, streams=[self.strm_f, self.xyrange]).opts(
                frame_height=int(500 * self._h / self._w), width=150, cmap="Viridis"
            )

            # add the histogram as an adjoint subfig
            im_ovly = im_ovly << his

            return im_ovly  # Return the overlay object

        # If layout is defined and metadata is available
        if self._layout and self.meta_dicts:
            im_dict = OrderedDict()  # Initialize an ordered dictionary to store the images
            for meta in itt.product(*list(self.meta_dicts.values())):  # For each combination of metadata values
                mdict = {k: v for k, v in zip(list(self.meta_dicts.keys()), meta)}  # Map each metadata key to its value
                im_dict[meta] = get_im_ovly(mdict)  # Add the generated overlay to the dictionary

            # Generate a HoloViews NdLayout object from the image dictionary
            ims = hv.NdLayout(im_dict, kdims=list(self.meta_dicts.keys()))
        else:
            # If no layout or metadata, generate an overlay for the current metadata
            ims = get_im_ovly(self.cur_metas)

        if self.summary is not None:  # If summary data is available
            # Generate a HoloViews Curve object from the summary data
            hvsum = (
                hv.Dataset(self.sum_sub)
                .to(hv.Curve, kdims=["frame"])
                .overlay("sum_var")
            )

            # Apply data shading if required
            if self._datashade:
                hvsum = datashade_ndcurve(hvsum, kdim="sum_var")

            try:
                hvsum = hvsum.layout(list(self.meta_dicts.keys()))  # Arrange the summary layout based on metadata
            except:
                pass

            # Generate a vertical line to indicate the current frame
            vl = hv.DynamicMap(lambda f: hv.VLine(f), streams=[self.strm_f]).opts(
                color="red")

            # Combine the summary curves and the vertical line, and apply dimensions and a colormap
            summ = (hvsum * vl).map(
                lambda p: p.opts(frame_width=500, aspect=3), [hv.RGB, hv.Curve]
            )

            # Combine the images and the summary into a single layout, arranged in columns
            hvobj = (ims + summ).cols(1)
        else:
            hvobj = ims  # If no summary data, the layout is just the images

        return hvobj  # Return the layout object


    def show(self) -> pn.layout.Column:
        # Return widgets and plots in a layout
        return pn.layout.Column(self.widgets, self.pnplot)

    def _widgets(self):
        w_play = pnwgt.Player(
            length=len(self._f), interval=1000//self.framerate, value=0, width=650, height=90
        )

        def play(f):
            if not f.old == f.new:
                self.strm_f.event(f=int(self._f[f.new]))

        w_play.param.watch(play, "value")
        w_box = pnwgt.Button(
            name="Update Mask", button_type="primary", width=100, height=30
        )
        w_box.param.watch(self._update_box, "clicks")
        if not self._layout:
            wgt_meta = {
                d: pnwgt.Select(name=d, options=v, height=45, width=120)
                for d, v in self.meta_dicts.items()
            }

            def make_update_func(meta_name):
                def _update(x):
                    self.cur_metas[meta_name] = x.new
                    self._update_subs()

                return _update

            for d, wgt in wgt_meta.items():
                cur_update = make_update_func(d)
                wgt.param.watch(cur_update, "value")
            wgts = pn.layout.WidgetBox(w_box, w_play, *list(wgt_meta.values()))
        else:
            wgts = pn.layout.WidgetBox(w_box, w_play)
        return wgts

    def _update_subs(self):
        self.ds_sub = self.ds.sel(**self.cur_metas)
        if self.sum_sub is not None:
            self.sum_sub = self.summary.sel(**self.cur_metas)
        self.pnplot.objects[0].object = self.get_hvobj()

    def _update_box(self, click):
        box = self.str_box.data
        self.mask.update(
            {
                tuple(self.cur_metas.values()): {
                    "height": slice(box["y0"][0], box["y1"][0]),
                    "width": slice(box["x0"][0], box["x1"][0]),
                }
            }
        )

def datashade_ndcurve(
    ovly: hv.NdOverlay, kdim: Optional[Union[str, List[str]]] = None, spread=False
) -> hv.Overlay:
    """
    Apply datashading to an overlay of curves with legends.

    Parameters
    ----------
    ovly : hv.NdOverlay
        The input overlay of curves.
    kdim : Union[str, List[str]], optional
        Key dimensions of the overlay. If `None` then the first key dimension of
        `ovly` will be used. By default `None`.
    spread : bool, optional
        Whether to apply :func:`holoviews.operation.datashader.dynspread` to the
        result. By default `False`.

    Returns
    -------
    hvres : hv.Overlay
        Resulting overlay of datashaded curves and points (for legends).
    """
    if not kdim:
        kdim = ovly.kdims[0].name
    var = np.unique(ovly.dimension_values(kdim)).tolist()
    color_key = [(v, Category10_10[iv]) for iv, v in enumerate(var)]
    color_pts = hv.NdOverlay(
        {
            k: hv.Points([0, 0], label=str(k)).opts(color=v)
            for k, v in color_key
        }
    )
    ds_ovly = datashade(
        ovly,
        aggregator=count_cat(kdim),
        color_key=dict(color_key),
        min_alpha=200,
        normalization="linear",
    )
    if spread:
        ds_ovly = dynspread(ds_ovly)
    return ds_ovly * color_pts

In [ ]:
vaviewer = VArrayViewer(data)
vaviewer.show()

# Scratch (ignore)

In [ ]:
# from typing import Optional, Dict, Union
# from pathlib import Path
# import os
# import xarray as xr
# import dask.array as darr
# import shutil

# # def save_minian(var: xr.DataArray, dpath: str, meta_dict: Optional[Dict[int, str]] = None,
# #     overwrite: bool = False, compute: bool = True,) -> Union[xr.DataArray, darr.Array]:
    
# #     """
# #     Function to save a xarray DataArray to a specified path as a zarr file.

# #     Parameters
# #     ----------
# #     var : xr.DataArray
# #         The input xarray DataArray that needs to be saved.
# #     dpath : str
# #         The destination path where the file needs to be saved.
# #     meta_dict : dict, optional
# #         A dictionary containing metadata. If provided, it's used to assign coordinates to the dataset. 
# #         The keys are names of the coordinates and the values are the indices to be used from the pathlist. 
# #         The default is None.
# #     overwrite : bool, optional
# #         If true, it will overwrite the file at the destination path, if one exists. The default is False.

# #     Returns
# #     -------
# #     Union[xr.DataArray, darr.Array]
# #         Returns a xarray DataArray if compute is True, otherwise a Dask array.
# #     """

# #     dpath = os.path.normpath(dpath)
# #     Path(dpath).mkdir(parents=True, exist_ok=True)
    
# #     # Convert the DataArray to a DataSet for saving (this would happen automatically anyway if we did DataArray.to_zarr)
# #     ds = var.to_dataset()
    
# #     # If there's a metadata dictionary, use it to assign coordinates to the dataset
# #     # This is useful to include some parts of the file path as metadata in the dataset itself, 
# #     # for instance, to indicate different experimental conditions
# #     if meta_dict is not None:
# #         # Get the list of directories in the path
# #         pathlist = os.path.split(os.path.abspath(dpath))[0].split(os.sep)
        
# #         # Assign the metadata items as coordinates to the dataset
# #         ds = ds.assign_coords(
# #             **dict([(dn, pathlist[di]) for dn, di in meta_dict.items()])
# #         )
    
# #     # Decide the write mode based on whether we're overwriting or not
# #     md = {True: "a", False: "w-"}[overwrite]
    
# #     # Set the full path to the zarr file
# #     fp = os.path.join(dpath, var.name + ".zarr")
    
# #     # If overwriting, try to remove the existing file first
# #     if overwrite:
# #         try:
# #             shutil.rmtree(fp)
# #         except FileNotFoundError:
# #             pass  # If the file wasn't there, no problem
    
# #     # Save the DataSet as a zarr file
# #     ds.to_zarr(fp, compute=compute, mode=md)
    
# #     # # If compute is true, open the saved zarr file, get the DataArray by name and replace the array data with the inlined zarr array
# #     # if compute:
# #     #     arr = xr.open_zarr(fp)[var.name]
# #     #     arr.data = darr.from_zarr(os.path.join(fp, var.name), inline_array=True)
    
# #     return